In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
train = pd.read_csv('yelp_train.csv')
test = pd.read_csv('yelp_test.csv')
sample = pd.read_csv('yelp_sample.csv')

In [2]:
def featurize_no_text(df):
    sent = pd.read_csv("vaderSentiment/vaderSentiment/vader_lexicon.txt", sep='\t', header=None).iloc[:, :2]
    sent.columns = ["token", "polarity"]
    sent.drop_duplicates(subset="token", keep='first',inplace=True)
    sent.set_index("token", inplace=True)
    df['authored_by_elite_user'] = [1 if x in y else 0 for x,y in zip(pd.to_datetime(df.date).dt.strftime('%Y'), df.user_elite)]
    df['user_business_avg_star_difference'] = df.user_average_stars-df.business_average_stars
    df["text"] = df["text"].str.lower()
    sentiment = []
    for text in np.arange(len(df)):
        sentiment.append(np.sum([sent.loc[i]["polarity"] for i in df.iloc[text,7].split() if i in sent.index]))
    df["sentiment"] = sentiment
    return df[['authored_by_elite_user', 'useful', 'cool', 'funny', 'user_average_stars', 'user_business_avg_star_difference', 'business_average_stars',"sentiment"]]

train_data = pd.read_csv('yelp_train.csv')
no_text_train = featurize_no_text(train_data)
no_text_train_y = train_data['is_good_rating']
no_text_test = featurize_no_text(pd.read_csv('yelp_test.csv'))

In [3]:
no_text_train.head()

,authored_by_elite_user,useful,cool,funny,user_average_stars,user_business_avg_star_difference,business_average_stars,sentiment
0,0,0,0,0,2.00,-2.00,4.0,1.5
1,0,0,0,0,4.43,0.93,3.5,3.3
2,0,0,0,0,4.09,0.09,4.0,3.1
3,0,0,1,0,3.55,0.55,3.0,15.2
4,0,1,1,1,3.75,0.25,3.5,-2.8


In [33]:
def featurize(data):
    sent = pd.read_csv("vaderSentiment/vaderSentiment/vader_lexicon.txt", sep='\t', header=None).iloc[:, :2]
    sent.columns = ["token", "polarity"]
    sent.drop_duplicates(subset="token", keep='first',inplace=True)
    sent.set_index("token", inplace=True)
    sentiment = []
    for text in np.arange(len(data)):
        sentiment.append(np.sum([sent.loc[i]["polarity"] for i in data.iloc[text,7].split() if i in sent.index]))
    data["text"] = data["text"].str.lower()
    a=np.zeros(len(data))
    b=np.zeros(len(data))
    c=np.zeros(len(data))
    d=np.zeros(len(data))
    e=np.zeros(len(data))
    for text in np.arange(len(data)):
        a[text]=data.iloc[text,7].count("!")
        b[text]=data.iloc[text,7].count("delicious")
        c[text]=data.iloc[text,7].count("useful")
        d[text]=data.iloc[text,7].count("great")
        e[text]=data.iloc[text,7].count("bad")
    df = pd.DataFrame(columns = ['sentiment',"!","delicious","useful","great","bad"]) 
    df.loc[:, "sentiment"] = sentiment
    df.loc[:, "!"] = a 
    df.loc[:, "delicious"] = b
    df.loc[:, "useful"] = c 
    df.loc[:, "great"] = d
    df.loc[:, "bad"] = e
    df.loc[:,'user_average_stars'] = data[['user_average_stars']]
    return df

In [34]:
train = pd.read_csv('yelp_train.csv')
X = featurize(train)
y = train['is_good_rating']
test = featurize(pd.read_csv('yelp_test.csv'))

In [23]:
X.head()

,sentiment
0,1.5
1,3.3
2,3.1
3,15.2
4,-2.8


In [37]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', solver='adam', learning_rate='adaptive', alpha=1e-4, hidden_layer_sizes=(100, 100, 100), random_state=42) 
X = X[["sentiment"]]
y = train['is_good_rating']
clf.fit(X, y)
predictions = clf.predict(test[["sentiment"]])
test_accuracy = np.mean(predictions==sample['is_good_rating'])
predictions_train = clf.predict(X)
train_accuracy = np.mean(predictions_train==y)
print("test: ", test_accuracy)
print("train: ", train_accuracy)

test:  0.7672333333333333
train:  0.7461833333333333


In [ ]:
predictions

In [38]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', solver='adam', learning_rate='adaptive', alpha=1e-4, hidden_layer_sizes=(150, 150, 100), random_state=42) 
X = df
y = train['is_good_rating']
clf.fit(X, y)
predictions = clf.predict(test)
accuracy = np.mean(predictions==sample['is_good_rating'])
predictions_train = clf.predict(X)
accuracy_train = np.mean(predictions_train==y)
print("test: ", test_accuracy)
print("train: ", train_accuracy)

test:  0.7672333333333333
train:  0.7461833333333333
